# Regresión Polinómica

### #Introducción, objetivo, y contexto#

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import sys
sys.path.append('././src')
from linear_regression import LinReg

In [10]:
# Cargamos los datos
Data = pd.read_csv('././data/dataset_regresion.csv', parse_dates=[0], date_format='%d.%m.%Y')

# Graficamos y echamos un vistazo a los datos
Data.plot(x='period', y='sales_quantity', xticks=[], style='--')
Data.head()

KeyError: 'period'

### estacionalidad, por qué es un problema y cómo se soluciona

In [ ]:
# Aplicamos una transformación básica de media móvil
Data['moving_avg'] = Data['sales_quantity'].shift(1).rolling(window=6, center=False).mean()
Data['deseasonalized'] = Data['sales_quantity'] - Data['moving_avg']

# Graficamos el resultado
Data.plot(x='period', y='deseasonalized', xticks=[], style='--')

## hay que agregar transformaciones polinómicas, probar y quedarnos con el que de el mejor score

In [ ]:
# Creamos variables dummy para los meses
month_dummies = pd.get_dummies(Data['period'].dt.month_name(), prefix='month', drop_first=True)
month_dummies = month_dummies.astype(int)
Data = pd.concat([Data, month_dummies], axis=1)

# Creamos las variables de rezago
lags = []
for i in range(1, 6):
    Data[f'Lag_{i}'] = Data['deseasonalized'].shift(i)
    lags.append(f'Lag_{i}')

# Eliminamos los valores nulos
Data.dropna(inplace=True)

In [ ]:
# Dividimos los datos en conjuntos de entrenamiento y prueba
train, test = train_test_split(Data, test_size=0.2, shuffle=False) # Mantener el orden temporal

# Definimos las características y el objetivo
features = lags + list(month_dummies.columns)
target = 'deseasonalized'

# Entrenamos el modelo
model = LinReg(verbose=True)
model.fit(train[features], train[target])

# Imprimimos el error cuadrático medio
print(f'Error cuadrático medio: {model.score(test[features], test[target])}')

In [ ]:
# Realizamos la predicción
test['y_pred'] = model.predict(test[features])

# Miro las predicciones
plt.plot(test["period"], test[target], '--', c='Blue', label='Real', alpha=0.4)
plt.plot(test["period"], test['y_pred'], '--', c='Red', label='Predicción')
plt.legend()
plt.show()

# Imprimo el resumen de las predicciones
test[['deseasonalized', 'Prediccion']].describe()

### observaciones y conclusiones